## Sentiment Analysis

In this exercise we use the IMDb-dataset, which we will use to perform a sentiment analysis. The code below assumes that the data is placed in the same folder as this notebook. We see that the reviews are loaded as a pandas dataframe, and print the beginning of the first few reviews.

In [40]:
import numpy as np
import pandas as pd

reviews = pd.read_csv('reviews.txt', header=None)
labels = pd.read_csv('labels.txt', header=None)
Y = (labels=='positive').astype(np.int_)

print(type(reviews))
print(reviews.head())

<class 'pandas.core.frame.DataFrame'>
                                                   0
0  bromwell high is a cartoon comedy . it ran at ...
1  story of a man who has unnatural feelings for ...
2  homelessness  or houselessness as george carli...
3  airport    starts as a brand new luxury    pla...
4  brilliant over  acting by lesley ann warren . ...


**(a)** Split the reviews and labels in test, train and validation sets. The train and validation sets will be used to train your model and tune hyperparameters, the test set will be saved for testing. Use the `CountVectorizer` from `sklearn.feature_extraction.text` to create a Bag-of-Words representation of the reviews. Only use the 10,000 most frequent words (use the `max_features`-parameter of `CountVectorizer`).

In [73]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.utils import to_categorical

labels = pd.get_dummies(labels)

# split to train and rest
X_train, X_rest, y_train, y_rest = train_test_split(reviews[0], labels, random_state=1)
# split rest to test and val
X_test, X_val, y_test, y_val = train_test_split(X_rest, y_rest, random_state=1)

# vectorize text
vectorizer = CountVectorizer(max_features=10000, lowercase=False)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
X_val = vectorizer.transform(X_val)  

**(b)** Explore the representation of the reviews. How is a single word represented? How about a whole review?

Each row of the matrix corresponds to a document, and each column corresponds to a word in the vocabulary. The matrix contains integer counts of the occurrences of each word in each document.

**(c)** Train a neural network with a single hidden layer on the dataset, tuning the relevant hyperparameters to optimize accuracy. 

In [118]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

from silence_tensorflow import silence_tensorflow
silence_tensorflow()

import logging
logging.getLogger('tensorflow').disabled = True

import numpy as np
from numpy.random import seed, randint



num_classes = 2
input_size = reviews[0].shape[0]

seed(0)
tf.random.set_seed(0)

def create_neuralnet(hidden_layer_size, activation, learning):
    model = Sequential() # initialize a neural network
    model.add(Dense(units = hidden_layer_size, activation=activation, input_shape=(10000,))) # add a hidden layer
    model.add(Dense(units = num_classes, activation='softmax')) #add the output layer

    sgd = optimizers.SGD(learning_rate = learning)
    model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

    return model

param_grid = {
    'hidden_layer_size': [32, 64, 96, 128, 256],
    'activation': ['relu', 'tanh'],
    'learning': [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3],
}

best_acc = 0
best_params = {}

for h in param_grid['hidden_layer_size']:
    for a in param_grid['activation']:
        for l in param_grid['learning']:
            model = create_neuralnet(h, a, l)
            accurracy = model.evaluate(X_test, y_test)[0]

            if(accurracy > best_acc):
                print(f'New best accurracy: {accurracy}')
                best_params = {
                        'hidden_layer_size': h,
                        'activation': a,
                        'learning': l,
                }
                best_acc = accurracy

print(f'Best accurracy found for params: {best_params}')


model = Sequential() # initialize a neural network
model.add(Dense(units = best_params['hidden_layer_size'], activation=best_params['activation'], input_shape=(10000,))) # add a hidden layer
model.add(Dense(units = num_classes, activation='softmax')) #add the output layer

sgd = optimizers.SGD(learning_rate = best_params['learning'])
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

history = model.fit(X_train, y_train, epochs = 40, batch_size = 50, validation_split = 0.2, verbose = 1)



/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step - accuracy: 0.5044 - loss: 0.7080
New best accurracy: 0.7118310928344727


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 715us/step - accuracy: 0.5056 - loss: 0.7076


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4803 - loss: 0.7471


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5105 - loss: 0.7114
Best accurracy found for params: {'hidden_layer_size': 256, 'activation': 'tanh', 'learning': 0.0001}
Epoch 1/40


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.4628 - loss: 0.7273 - val_accuracy: 0.5013 - val_loss: 0.7076
Epoch 2/40
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5256 - loss: 0.6985 - val_accuracy: 0.5557 - val_loss: 0.6879
Epoch 3/40
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5650 - loss: 0.6797 - val_accuracy: 0.5869 - val_loss: 0.6729
Epoch 4/40
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5901 - loss: 0.6651 - val_accuracy: 0.6077 - val_loss: 0.6610
Epoch 5/40
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6141 - loss: 0.6530 - val_accuracy: 0.6221 - val_loss: 0.6509
Epoch 6/40
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6292 - loss: 0.6428 - val_accuracy: 0.6333 - val_loss: 0.6422
Epoch 7/40
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6379 - loss: 0.6337 - val_accuracy: 0.6392 - val_loss: 0.6344
Epoch 8/40
300/300 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6487 - loss: 0.6256 - val_accuracy: 0.6469 - val_

**(d)** Test your sentiment-classifier on the test set.

In [119]:
print("Loss + accuracy on test data: {}".format(model.evaluate(X_test, y_test)))

147/147 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7769 - loss: 0.5076
Loss + accuracy on test data: [0.5111280083656311, 0.7740558981895447]


**(e)** Use the classifier to classify a few sentences you write yourselves. 

In [120]:
# Index 0 is negative, index 1 is positive
sentence1 = vectorizer.transform(['i hated it'])
sentence2 = vectorizer.transform(['adam sandler'])
sentence3 = vectorizer.transform(['i loved it'])
sentence4 = vectorizer.transform(['cilian murphy'])
sentence5 = vectorizer.transform(['brad pitt'])
sentence6 = vectorizer.transform(['gay'])



def log_model_outcome(res):
    if(res[0][0] > 0.5):
        print('The sentence is most likely negative')
    else:
        print('The sentence is most likely positive')

log_model_outcome(model.predict(sentence1))
log_model_outcome(model.predict(sentence2))
log_model_outcome(model.predict(sentence3))
log_model_outcome(model.predict(sentence4))
log_model_outcome(model.predict(sentence5))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
The sentence is most likely positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
The sentence is most likely positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
The sentence is most likely positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
The sentence is most likely positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
The sentence is most likely positive
